In [3]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer
from global_vars import *
import torch

model_saved = "mbti_bert_small.pt"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMultipleChoice.from_pretrained(model_name)
model.load_state_dict(torch.load(model_saved, map_location=torch.device('cpu')))
model.eval();

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMultipleChoice: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [4]:
from torch.nn import functional as F
#input_text = "All things in moderation  Sims is indeed a video game, and a good one at that Note a good one at that is somewhat subjective in that I am not completely promoting the death of any given Sim"
#input_text = "No Its like a theme for where I live and that is why I know it by heart"
input_text = "I collect shoes I do so because I like status and nothing communicates such a thing as much as a pair of Jordans"
choices = ["ISTJ", "ISFJ", "INFJ", "INTJ", "ISTP", "ISFP", "INFP", "INTP", "ESTP", "ESFP", "ENFP", "ENTP", "ESTJ", "ESFJ", "ENFJ", "ENTJ"]

map_val = {0: "ISTJ", 1: "ISFJ", 2: "INFJ", 3: "INTJ", 4: "ISTP", 5: "ISFP", 6: "INFP", 7: "INTP", 8: "ESTP", 9: "ESFP", 10: "ENFP", 11: "ENTP", 12: "ESTJ", 13: "ESFJ", 14: "ENFJ", 15: "ENTJ"}


encoding = tokenizer([input_text] * 16, choices, return_tensors="pt", padding=True, truncation=True)
output = model(**{k: v.unsqueeze(0) for k, v in encoding.items()})

logits = output.logits
print(logits)

softmax = F.softmax(logits, dim = -1)
top_5 = torch.topk(softmax, 5, dim = 1)[1][0]
for i in top_5:
    print(map_val[i.item()])



tensor([[-5.1153, -5.1847,  3.5397,  2.5313, -5.1232, -5.1992,  2.1180,  3.2202,
         -5.1268, -5.2320, -5.3541, -2.7298, -5.1565, -5.1948, -1.5902, -2.8993]],
       grad_fn=<ViewBackward0>)
INFJ
INTP
INTJ
INFP
ENFJ
